In [5]:
# CHANGE INFO TO RUN ON YOUR LOCAL
host = "127.0.0.1"
port = "5432"
database = "MSDS691"
# database = 'msds691'
user = "postgres"


In [58]:
import psycopg2
import pandas as pd
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML

conn = psycopg2.connect(host=host, port=port, database=database, user=user)
cur = conn.cursor()


In [7]:
def select_query(query):
    df = pd.read_sql_query(query, conn, coerce_float=False)
    df.replace([None], np.nan, inplace=True)

    return df


In [8]:
from IPython.core.display import display, HTML


def display_side_by_side(dfs: list, captions: list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """

    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'")\
                    .set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))


In [9]:
cur.execute(f'''DROP TABLE IF EXISTS names;''')
cur.execute(f'''DROP TABLE IF EXISTS transactions;''')


In [10]:
create = f'''

DROP TABLE IF EXISTS names;

CREATE TABLE names
(id INTEGER,
name VARCHAR,
PRIMARY KEY (id))
;

INSERT INTO names
VALUES
(1, 'Jon Smith'),
(2, 'Sarah Adams'),
(3, 'Maria Lopez')
;'''

cur.execute(create)
conn.commit()

create = f'''

DROP TABLE IF EXISTS transactions;

CREATE TABLE transactions
(id INTEGER,
 amount NUMERIC,
 PRIMARY KEY (id))
;

INSERT INTO transactions
VALUES
(1, 10),
(3, 20),
(7, 50)
;'''

cur.execute(create)
conn.commit()


In [11]:
query = f'''

SELECT *
FROM names

;'''
df_names = select_query(query)
df_names


,id,name
0,1,Jon Smith
1,2,Sarah Adams
2,3,Maria Lopez


In [12]:
query = f'''

SELECT *
FROM transactions

;'''
df_transactions = select_query(query)
df_transactions


,id,amount
0,1,10
1,3,20
2,7,50


### Inner Join

In [13]:
query = f'''

SELECT  names.id, names.name, transactions.amount
FROM names INNER JOIN transactions
    ON (names.id = transactions.id)

;'''
df_inner_join = select_query(query)
display_side_by_side([df_names, df_transactions, df_inner_join],
                     ['Left Table', 'Right Table', 'Inner Join Output'])


### Left Join

In [14]:
query = f'''

SELECT  names.id, names.name, transactions.amount
FROM names LEFT JOIN transactions
    ON (names.id = transactions.id)

;'''
df_left_join = select_query(query)
display_side_by_side([df_names, df_transactions, df_left_join],
                     ['Left Table', 'Right Table', 'Left Join Output'])


### Right Join

In [15]:
query = f'''

SELECT  transactions.id, names.name, transactions.amount
FROM names RIGHT JOIN transactions
    ON (names.id = transactions.id)

;'''
df_right_join = select_query(query)
display_side_by_side([df_names, df_transactions, df_right_join],
                     ['Left Table', 'Right Table', 'Right Join Output'])


### Full Join

In [16]:
query = f'''

SELECT COALESCE(names.id,transactions.id) as id,
       names.name, transactions.amount
FROM names FULL JOIN transactions
    ON (names.id = transactions.id)

;'''
df_full_join = select_query(query)
display_side_by_side([df_names, df_transactions, df_full_join],
                     ['Left Table', 'Right Table', 'Full Join Output'])


### Cross Join

In [21]:
query = f'''

SELECT *
FROM names CROSS JOIN transactions

;'''
df_cross_join = select_query(query)
df_cross_join


,id,name,id,amount
0,1,Jon Smith,1,10
1,1,Jon Smith,3,20
2,1,Jon Smith,7,50
3,2,Sarah Adams,1,10
4,2,Sarah Adams,3,20
5,2,Sarah Adams,7,50
6,3,Maria Lopez,1,10
7,3,Maria Lopez,3,20
8,3,Maria Lopez,7,50


### Add dob_table

In [22]:
create = f'''
DROP TABLE IF EXISTS dob_table;


CREATE TABLE dob_table
(id INTEGER,
 dob date,
 PRIMARY KEY (id));

INSERT INTO dob_table
VALUES
(1, '1982-09-29'),
(3, '1996-02-16')
;'''

cur.execute(create)
conn.commit()


### Select all dob_table

In [23]:
query = f'''

SELECT *
FROM dob_table

;'''
df_dob = select_query(query)
df_dob


,id,dob
0,1,1982-09-29
1,3,1996-02-16


### Join 3 tables

In [24]:
query = f'''

SELECT  names.id, names.name, transactions.amount, dob_table.dob
FROM names
    INNER JOIN transactions
        ON (names.id = transactions.id)
    INNER JOIN dob_table
        ON (names.id = dob_table.id)

;'''
df_three_tables = select_query(query)
display_side_by_side([df_names, df_transactions, df_dob, df_three_tables],
                     ['names', 'transactions', 'dob', 'all three tables'])


,id,name
0,1,Jon Smith
1,2,Sarah Adams
2,3,Maria Lopez
,id,amount
0,1,10
1,3,20
2,7,50
,id,dob
0,1,1982-09-29
1,3,1996-02-16





### One-to-One | One-to-Many | Many-to-Many

In [29]:
Image(url= "one_to_one.png")

In [59]:
create = f'''

DROP TABLE IF EXISTS long_transactions;

CREATE TABLE long_transactions
(id INTEGER,
amount NUMERIC);

INSERT INTO long_transactions
VALUES
(1, 45),
(1, 10),
(2, 10),
(3, 20),
(3, 50),
(7, 50)
;'''
cur.execute(create)
conn.commit()


### long_transactions table

In [60]:
query = f'''

SELECT  *
FROM long_transactions

;'''
df_long_tran = select_query(query)
df_long_tran


,id,amount
0,1,45
1,1,10
2,2,10
3,3,20
4,3,50
5,7,50


### One-to-many

In [61]:
query = f'''

SELECT  
names.id, 
names.name, 
long_transactions.amount
FROM names 
    LEFT JOIN long_transactions
        ON (names.id = long_transactions.id)

;'''
df_one_many = select_query(query)
display_side_by_side([df_names, df_long_tran, df_one_many],
                     ['names', 'transactions', 'One-to-many'])


,id,name
0,1,Jon Smith
1,2,Sarah Adams
2,3,Maria Lopez
,id,amount
0,1,45
1,1,10
2,2,10
3,3,20
4,3,50
5,7,50


### Many-to-Many

In [62]:
create = f'''

DROP TABLE IF EXISTS long_amounts;

CREATE TABLE long_amounts
(amount NUMERIC,
 item VARCHAR);

INSERT INTO long_amounts
VALUES
(10, 'toy'),
(10, 'postcard'),
(50, 'shirt'),
(20, 'cards'),
(20, 'other cards'),
(70, 'pants')
;'''
cur.execute(create)
conn.commit()

### long_amounts

In [54]:
query = f'''

SELECT  *
FROM long_amounts

;'''
df_long_amount = select_query(query)
df_long_amount


,amount,item
0,10,toy
1,10,postcard
2,50,shirt
3,20,cards
4,20,other cards
5,70,pants


In [40]:
conn.rollback()

In [63]:
query = f'''

SELECT  
long_transactions.id,
long_transactions.amount,
long_amounts.item

FROM long_transactions 
    INNER JOIN long_amounts
        ON (long_transactions.amount = long_amounts.amount)

;'''
df_many_many = select_query(query)
display_side_by_side([df_long_tran, df_long_amount, df_many_many],
                     ['long_transactions', 'long_amounts', 'Many-to-Many'])


,id,amount
0,1,45
1,1,10
2,2,10
3,3,20
4,3,50
5,7,50
,amount,item
0,10,toy
1,10,postcard
2,50,shirt


In [64]:
cur.close()
conn.close()
